In [35]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.preprocessing import KBinsDiscretizer, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from causalml.inference.meta import BaseTClassifier
import seaborn as sns

from causalml.dataset import make_uplift_classification, synthetic_data
from causalml.inference.tree import UpliftRandomForestClassifier


In [37]:
df, X_names = make_uplift_classification(
    n_samples=10000, treatment_name=["control", "treatment1", "treatment2"]
)

In [38]:
y = "conversion"
T = "treatment_group_key"
control = 0
X = [col for col in df.columns if col not in [y, "conversion"] and "informative" in col]

In [39]:
T_encoder = OrdinalEncoder()
df[T] = T_encoder.fit_transform(df[T].to_numpy().reshape(-1, 1)).astype(int)
T_encoder = OrdinalEncoder()
df[T] = T_encoder.fit_transform(df[T].to_numpy().reshape(-1, 1)).astype(int)


In [40]:
df['value'] = 20

In [41]:
train, test = train_test_split(df)


In [42]:
cost_dict = {0: 0, 1: 5, 2: 10}

In [80]:
def XLearner():
    # first stage models

    response_models = {}
    for t in train[T].unique():
        m = LGBMClassifier(max_depth=2, min_child_samples=30)
        m.fit(train.query(f"{T}=={t}")[X], train.query(f"{T}=={t}")[y])
        response_models[t] = m

    # propensity model
    e = LGBMClassifier(solver="lbfgs", penalty="none")
    e.fit(train[X], train[T])

    psuedo_models = {}
    for t in train[T].unique():
        mj = response_models[t]
        m0 = response_models[control]

        control_df = train[(train[T] == control)]
        treatment_df = train[(train[T] == t)]

        s_t0 = np.ones(control_df.shape[0]) * cost_dict[control]
        s_tj = np.ones(treatment_df.shape[0]) * cost_dict[t]

        ic_t0 = np.ones(control_df.shape[0]) * 0.1
        ic_tj = np.ones(treatment_df.shape[0]) * 0.1

        D_t0 = (control_df["value"] - s_t0) * (
            mj.predict(control_df[X]) - control_df['value'] * control_df[y]
        ).to_numpy() - ic_t0

        D_tj = (treatment_df["value"] - s_tj) * (
            treatment_df[y] - treatment_df['value'] * m0.predict(treatment_df[X])
        ).to_numpy() - ic_tj

        t0_effect_key = f"{control}_{t}"
        tj_effect_key = f"{t}_{control}"

        psuedo_models[t0_effect_key] = LGBMRegressor(
            max_depth=2, min_child_samples=30
        ).fit(train[train[T] == control][X], D_t0)

        psuedo_models[tj_effect_key] = LGBMRegressor(
            max_depth=2, min_child_samples=30
        ).fit(train[train[T] == t][X], D_tj)

    probs = e.predict_proba(train[X])

    cate_tracker = {}
    for t in train[T].unique():
        if t == control:
            continue
        tau0_key = f"{control}_{t}"
        tauj_key = f"{t}_{control}"

        denom = probs[:, control] + probs[:, t]
        cate = probs[:, t] / denom * psuedo_models[tau0_key].predict(train[X]) + probs[
            :, control
        ] * psuedo_models[tauj_key].predict(train[X])
        cate_tracker[t] = cate

    train_preds = pd.DataFrame(cate_tracker)
    train_preds[0] = 0

    probs = e.predict_proba(test[X])

    cate_tracker = {}
    for t in test[T].unique():
        if t == control:
            continue
        tau0_key = f"{control}_{t}"
        tauj_key = f"{t}_{control}"

        denom = probs[:, control] + probs[:, t]
        cate = probs[:, t] / denom * psuedo_models[tau0_key].predict(test[X]) + probs[
            :, control
        ] * psuedo_models[tauj_key].predict(test[X])
        cate_tracker[t] = cate

    test_preds = pd.DataFrame(cate_tracker)
    test_preds[0] = 0

    # x_cate_test = cate_tracker.max(axis=1)
    # treatment_test = cate_tracker.idxmax(axis=1)
    return train_preds, test_preds

In [81]:
train_preds, test_preds = XLearner()
X_best_ite = test_preds.idxmax(axis=1)
X_best_ite_train = train_preds.idxmax(axis=1)


In [69]:
test_preds.index = test.index

In [71]:
test['trigger_cost'] = test.treatment_group_key.apply(lambda x: cost_dict[x])
test['actual_value'] = (test['value'] - test['trigger_cost']) * test['conversion'] - 0.1

In [73]:
test[test.treatment_group_key == test_preds.idxmax(axis=1)].actual_value.mean()

10.008173076923077

In [77]:
random_preds = pd.Series(np.random.choice([0, 1, 2], test.shape[0]), index=test.index)
test[test.treatment_group_key == random_preds].actual_value.mean()

7.83843661401377